# Using Recursive Functions to Scan the File System
by César Pérez


## Introduction. 

In this article, I'll present an example to scan and extract information from our file system using [recursive functions](https://www.w3schools.com/python/gloss_python_function_recursion.asp). We'll define our goal as to:

* Iterate each file/directory starting from an specified root path inside our file system. 
* Save the filename and size of each element inside a dictionary.
* Format our data in diverse ways to be used by different components.
* Create visual representations of our data.
* Display these visual representations inside an HTML report for our final users. 

## Step 1. Create functions.

### File_Scan

This funtion will scan our file system by using os.list() to extract the content of a target folder and recursing when finding a subdirectory. For the purposes of this article, I will exclude everything on my .git folder.

In [ ]:
import os
import pandas as pd
import re 
from treelib import Node, Tree
from operator import itemgetter

#creates the file system map
def file_scan(root_dir):
    current_path = os.listdir(root_dir)
    file_count = 0 
    file_list = []
    dir_count = 0
    dir_dict = {}
    
    for element in current_path:
        next_path = os.path.join(root_dir, element)
        if os.path.isdir(next_path) and element != '.git':
            dir_count += 1
            dir_dict[element] = file_scan(next_path)
        elif element != '.git':
            file_count += 1
            file_list.append({'name':element, 'size': os.stat(next_path).st_size})
        dir_dict['total_files'] = file_count
        dir_dict['total_directories'] = dir_count
        dir_dict['files'] = file_list
    return dir_dict

### create_tree and create_node

These two funtions will help us to create a tree-like representation of the file system, the library we are using is [treelib](https://treelib.readthedocs.io/en/latest/). 

In [ ]:
def create_tree(dir_dict):
    tree = Tree()
    tree.create_node("*", "root")
    for element in dir_dict:
        if type(dir_dict[element]) == dict:
            tree.create_node(element,  element, parent="root")
            create_node(dir_dict[element], element, tree)
        elif type(dir_dict[element]) == list:
            for list_element in dir_dict[element]:
                tree.create_node(list_element['name'],  list_element['name'], parent="root")
    return tree

#Fuction complementary to create_tree()
def create_node(child_dict,level, tree):
    for element in child_dict:
        if type(child_dict[element]) == dict:
            tree.create_node(element,  element, parent=level)
            create_node(child_dict[element], element, tree)
        elif type(child_dict[element]) == list:
            for list_element in child_dict[element]:
                tree.create_node(list_element['name'],  list_element['name'], parent=level)
    return

### creates_summary and append_summary

These funtions will create two outputs:

1. Pandas dataframe parsed as HTML.
2. An array that will be parsed by a JS dynamic chart (we will further explore this later).

In [ ]:
def creates_summary(dir_dict):
    summary_df = pd.DataFrame()
    summary_count = {}
    summary_size = {}
    pie_chart_data = []
    for element in dir_dict:
        if type(dir_dict[element]) == dict:
            append_summary(dir_dict[element], summary_count, summary_size)
        elif type(dir_dict[element]) == list:
            for list_element in dir_dict[element]:
                file_ext = re.search('[\w ]*\.([\w]*)', list_element['name']).group(1)
                summary_count[file_ext] = summary_count.get(file_ext,0) + 1
                summary_size[file_ext] = summary_size.get(file_ext,0) + (list_element['size']/1000) #Represents KB
    
    for element in summary_count:
        summary_df.loc[element, 'count'] = summary_count[element]
        summary_df.loc[element, 'size_KB'] = round(summary_size[element],2)
        pie_chart_data.append([element, summary_size[element]])
    
    pie_chart_data = sorted(pie_chart_data, key=itemgetter(1), reverse=True)
    pie_chart_data.insert(0, ['File type', 'Size'])

    summary_df['size_%'] = round((summary_df['size_KB'].astype(float) / summary_df['size_KB'].sum()) *100,2)
    summary_df = summary_df.sort_values(by=['size_%'], ascending=False)
    return summary_df.to_html(justify = 'center'), pie_chart_data

#Fuction complementary to creates_summary()
def append_summary(child_dict, summary_count, summary_size):
    for element in child_dict:
        if type(child_dict[element]) == dict:
            append_summary(child_dict[element], summary_count, summary_size)
        elif type(child_dict[element]) == list:
            for list_element in child_dict[element]:
                file_ext = re.search('[\w ]*\.([\w]*)', list_element['name']).group(1)
                summary_count[file_ext] = summary_count.get(file_ext,0) + 1
                summary_size[file_ext] = summary_size.get(file_ext,0) + (list_element['size']/1000) #Represents KB
    return

## Step 2. Generate Data

Now that our functions have been declared, its time to execute them to gather the data about our file system we want to explore and share. Each of the following three cells will print the final outcome so we have a better understanding of what will be sending to the html report.

In [ ]:
#1. gets notebook dir
dirname = os.getcwd()

#2. Creates main dict to create objects
main_dict = file_scan(os.path.join(dirname, '..//..'))
main_dict

In [ ]:
#3. File tree
tree_filename = os.path.join(dirname, 'tree.txt')
file_tree = create_tree(main_dict)

file_tree.save2file(tree_filename)
tree_txt_reader = open(tree_filename, 'r', encoding='utf-8')
tree_txt = ""

for line in tree_txt_reader.readlines():
    line = '<pre>'+line+'</pre>'
    tree_txt += line

tree_txt_reader.close()
os.remove(tree_filename)
tree_txt

In [ ]:
#4. Pie Chart and table
table_summary, pie_chart = creates_summary(main_dict)
table_summary

In [ ]:
pie_chart

## Step 3. Create HTML report

For the purposes of this article, I've decided to use a simple approach. I've created an [empty template](template.html), then python will read this file, and insert the system data by using placeholders. Finally, the result is written into a different file.

In [ ]:
# #5. Create report
template_filename = os.path.join(dirname, 'template.html')

template_reader = open(template_filename, 'r')
template_content = template_reader.read()
template_reader.close()

new_content = re.sub("pie_chart_data_goes_here", str(pie_chart), template_content)
new_content = re.sub("table_goes_here", str(table_summary), new_content)
new_content = re.sub("file_tree_goes_here", tree_txt, new_content)

report_filename = os.path.join(dirname, 'report.html')
template_writer = open(report_filename, 'w', encoding="utf-8")
template_writer.write(new_content)

4725

The dashboard, at the point of uploading this notebook is quite simple, some aspects to keep in mind:
* Three divs, each having one of our objects.
* css is declared at the head tag.
* The pie chart is build using Google Charts, you can find the documentation [here](https://developers.google.com/chart/interactive/docs/gallery/piechart).

# [Here is the final report!](report.html)